# Fuzzing with Constraints

In this chapter, we show how to extend grammars with _functions_ – pieces of code that get executed during grammar expansion, and that can generate, check, or change elements produced.  Adding functions to a grammar allows for very versatile test generation, bringing together the best of grammar generation and programming.

**Prerequisites**

* As thus chapter deeply interacts with the techniques discussed in the [chapter on efficient grammar fuzzing](GrammarFuzzer.ipynb), a good understanding of the techniques is recommended.

## Example: Test a Credit Card System

Suppose you work with a shopping system that – among several other features – allows customers to pay with a credit card.  Your task is to test the payment functionality.  

To make things simple, we will simply assume that we need to pieces of data – a 16-digit credit card number and an amount to be charged.  Both pieces can be easily generated with grammars, as in the following:

In [ ]:
import fuzzingbook_utils

In [ ]:
from Grammars import EXPR_GRAMMAR, is_valid_grammar, is_nonterminal, opts, exp_opt, exp_string

In [ ]:
CHARGE_GRAMMAR = {
    "<start>": ["Charge <amount> to my credit card <credit-card-number>"],
    "<amount>": ["$<float>"],
    "<float>": ["<integer>.<digit><digit>"],
    "<integer>": ["<digit>", "<integer><digit>"],
    "<digit>": crange('0', '9'),

    "<credit-card-number>": ["<digits>"],
    "<digits>": ["<digit-block><digit-block><digit-block><digit-block>"],
    "<digit-block>": ["<digit><digit><digit><digit>"],
}

assert is_valid_grammar(CHARGE_GRAMMAR)

All of this works neatly – we can generate arbitrary amounts and credit card numbers:

In [ ]:
from GrammarFuzzer import GrammarFuzzer, all_terminals

In [ ]:
g = GrammarFuzzer(CHARGE_GRAMMAR)
[g.fuzz() for i in range(5)]

However, when actually testing our system with this data, we find two problems:

1. We'd like to test _specific_ amounts being charged – for instance, amounts that would excess the credit card limit.
2. We find that 9 out of 10 credit card numbers are rejected because of having an incorrect checksum.  This is fine if we want to test rejection of credit card numbers – but if we want to test the actual functionality of processing a charge, we need _valid_ numbers.

We could go and ignore these issues; after all, eventually, it is only a matter of time until large amounts and valid numbers are generated.  As it comes to the first concern, we could also address it by changing the grammar appropriately – say, to only produce charges that have at least six leading digits.  However, generalizing this to arbitrary ranges of values will be cumbersome.

The second concern, the checksums of credit card numbers, however, runs deeper – at least as far as grammars are concerned, is that a complex arithmetic operation like a checksum cannot be expressed in a grammar alone – at least not in the _context-free grammars_ we use here.  (In principle, one _could_ do this in a  _context–sensitive_ grammar, but specifying this would be is no fun at all.)  What we want is a mechanism that allows us to _attach programmatic computations_ to our grammars, bringing together the best of both worlds.

## Attaching Functions to Expansions

The key idea of this chapter is to _extend_ grammars such that one can _attach Python functions_ to individual expansions.  These functions can be executed 

1. _before_ expansion, _replacing_ the element to be expanded by a computed value; or
2. _after_ expansion, _checking_ generated elements, and possibly also replacing them.

In both cases, functions are specified using the `opts()` expansion mechanism introduced in the [chapter on grammars](Grammars.ipynb).  They are thus tied to a specific expansion $e$ of a symbol $s$.

### Functions Called Before Expansion

A function defined using the `pre` option is invoked _before_ expansion of $s$ into $e$. Its value _replaces_ the expansion $e$ to be produced.  To generate a value for the credit card example, above, we could define a _pre-expansion_ generator function

In [ ]:
import random

In [ ]:
def high_charge():
    return random.randint(10000000, 90000000) / 100.0

In [ ]:
high_charge()

With `opts()`, we could attach this function to the grammar:

In [ ]:
CHARGE_GRAMMAR.update({
    "<float>": [("<integer>.<digit><digit>", opts(pre=high_charge))],
})

with the intention that whenever `<float>` is expanded, the function `high_charge` would be invoked to generate a value for `<float>`.

Since functions tied to a grammar are frequently very simple, we can also _inline_ them using a *lambda* expression.  A _lambda expression_ is used for _anonymous_ functions that are limited in scope and functionality.  Here's an example:

In [ ]:
def apply_twice(function, x):
    return function(function(x))

apply_twice(lambda x: x * x, 2)

Here, we don't have to give the `function` to be applied twice a name (say, `square()`); instead, we apply it inline within the invocation.

Using `lambda`, this is what our grammar looks like:

In [ ]:
CHARGE_GRAMMAR.update({
    "<float>": [("<integer>.<digit><digit>", 
                 opts(pre=lambda: random.randint(10000000, 90000000) / 100.0))]
})

### Functions Called After Expansion

A function defined using the `post` option is invoked _after_ expansion of $s$ into $e$, passing the expanded values of the symbols in $e$ as arguments.  A post-expansion function can serve in two ways:

1. It can serve as a *constraint* or _filter_ on the expanded values, returning `True` if the expansion is valid, and `False` if not; if it returns `False`, another expansion is attempted.
2. It can also serve as a *repair*, returning a string value; like pre-expansion functions, the returned value replaces the expansion.

For our credit card example, we can chose both ways.  If we have a function `check_credit_card(s)` which returns `True` for a valid number `s` and `False` for invalid ones, we would go for the first option:

In [ ]:
CHARGE_GRAMMAR.update({
    "<credit-card-number>": [("<digits>", opts(post=lambda digits: check_credit_card(digits)))]
})

With such a filter, only valid credit cards will be produced.  On average, it will still take 10 attempts for each time `check_credit_card()` is satisfied, but then, we do not have to recourse to the system under test.

If we have a function `fix_credit_card(s)` which changes the number such that the checksum is valid and returns the "fixed" number, we can make use of this one instead:

In [ ]:
CHARGE_GRAMMAR.update({
    "<credit-card-number>": [("<digits>", opts(post=lambda digits: fix_credit_card(digits)))]
})

Here, each number is generated only once and then repaired.  This is very efficient.

The checksum function used for credit cards is the [Luhn algorithm](https://en.wikipedia.org/wiki/Luhn_algorithm), a simple yet effective formula.

In [ ]:
def luhn_checksum(s):
    """Compute Luhn's check digit over a string of digits"""
    LUHN_ODD_LOOKUP = (0, 2, 4, 6, 8, 1, 3, 5, 7, 9)  # sum_of_digits (index * 2)
    
    evens = sum(int(p) for p in s[-1::-2])
    odds = sum(LUHN_ODD_LOOKUP[int(p)] for p in s[-2::-2])
    return (evens + odds) % 10

In [ ]:
def valid_luhn_checksum(s):
    """Check whether the last digit is Luhn's checksum over the earlier digits"""
    return luhn_checksum(s[:-1]) == int(s[-1])

def fix_luhn_checksum(s):
    """Return the given string of digits, with a fixed check digit"""
    return s[:-1] + repr(luhn_checksum(s[:-1]))

In [ ]:
luhn_checksum("123")

In [ ]:
fix_luhn_checksum("123x")

We can make use of these functions in our credit card grammar:

In [ ]:
check_credit_card = valid_luhn_checksum
fix_credit_card = fix_luhn_checksum

fix_credit_card("1234567890123456")

## A Class for Integrating Constraints

While it is easy to specify functions, our grammar fuzzer will simply ignore them just as it ignores all extensions.  It will issue a warning, though:

In [ ]:
g = GrammarFuzzer(CHARGE_GRAMMAR)
g.fuzz()

We need to define a special fuzzer that actually invokes the given `pre` and `post` functions and acts accordingly.  We name this a `ConstraintGrammarFuzzer`:

In [ ]:
class ConstraintGrammarFuzzer(GrammarFuzzer):
    def supported_opts(self):
        return super().supported_opts() | {"pre", "post", "order"}

We define custom functions to access the `pre` and `post` options:

In [ ]:
def exp_pre_expansion_function(expansion):
    """Return the specified pre-expansion function, or None if unspecified"""
    return exp_opt(expansion, 'pre')

In [ ]:
def exp_post_expansion_function(expansion):
    """Return the specified post-expansion function, or None if unspecified"""
    return exp_opt(expansion, 'post')

## Generating Elements before Expansion

Our first task will be implementing the pre-expansion functions – that is, the function that would be invoked _before_ expansion to replace the value to be expanded.  To this end, we extend the `expansion_to_children()` method, whose task it is to convert an expansion string into a list of children subtrees.  We set it up such that it invokes the given `pre` function and applies its result on the children, possibly replacing them.

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def expansion_to_children(self, expansion):
        children = super().expansion_to_children(expansion)
        function = exp_pre_expansion_function(expansion)
        if function is None:
            return children
        
        assert callable(function)
        result = function()

        if self.log:
            print(repr(function) + "()", "=", repr(result))
        return self.apply_result(result, children)

The method `apply_result()` takes the result from the pre-expansion function and applies it on the children.  The exact effect depends on the type of the result:

* A _string_ $s$ replaces the entire expansion with $s$.
* A _list_ $(x_1, x_2, \dots, x_n)$ replaces the $i$-th symbol with $x_i$ for every $x_i$ that is not `None`.  Specifying `None` as a list element $x_i$ is useful to leave that element unchanged.  If $x_i$ is not a string, it is converted to a string.
* A value of `None` is ignored.  This is useful if one wants to simply call a function upon expansion, with no effect on the expanded strings.
* _Boolean_ values are ignored.  This is useful for post-expansion functions, discussed below.
* All _other types_ are converted to strings, replacing the entire expansion.

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def apply_result(self, result, children):
        if isinstance(result, str):
            children = [(result, [])]
        elif isinstance(result, list):
            symbol_indexes = [i for i, c in enumerate(children) if is_nonterminal(c[0])]

            for index, value in enumerate(result):
                if value is not None:
                    child_index = symbol_indexes[index]
                    if not isinstance(value, str):
                        value = repr(value)
                    if self.log:
                        print("Replacing", all_terminals(children[child_index]), "by", value)

                    # children[child_index] = (value, [])
                    (child_symbol, _) = children[child_index]
                    children[child_index] = (child_symbol, [(value, [])])
        elif result is None:
            pass
        elif isinstance(result, bool)
            pass
        else:
            if self.log:
                print("Replacing", "".join([all_terminals(c) for c in children]), "by", result)

            children = [(repr(result), [])]

        return children

With this, we have full support for pre-expansion functions.  Using the augmented `CHARGE_GRAMMAR`, we find that the pre-expansion `lambda` function is actually used:

In [ ]:
charge_fuzzer = ConstraintGrammarFuzzer(CHARGE_GRAMMAR)
charge_fuzzer.fuzz()

The log reveals a bit more details what happens when the pre-expansion function is called.  We see that the expansion `<integer>.<digit><digit>` is directly replaced by the computed value:

In [ ]:
amount_fuzzer = ConstraintGrammarFuzzer(CHARGE_GRAMMAR, start_symbol="<amount>", log=True)
amount_fuzzer.fuzz()

We can use such pre-expansion functions in other contexts, too.  Suppose we want to generate arithmetic expressions in which each number is between 100 and 200.  We can extend `EXPR_GRAMMAR` accordingly:

In [ ]:
import copy

In [ ]:
constrained_expr_grammar = copy.deepcopy(EXPR_GRAMMAR)

constrained_expr_grammar.update(
    {
     "<start>": ["<expr>"],
     "<factor>": ["+<factor>", "-<factor>", "(<expr>)",
         # Generate only the integer part with a function;
         # the fractional part comes from the grammar
         ("<integer>.<integer>", opts(pre=lambda: [random.randint(100, 200), None])),
         # Generate the entire integer from the function
         ("<integer>", opts(pre=lambda: random.randint(100, 200))),
        ],
    }
)

In [ ]:
expr_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
expr_fuzzer.fuzz()

## Checking Elements after Expansion

With this, we can now turn to our second set of functions to be supported.

In [ ]:
constrained_expr_grammar = copy.deepcopy(EXPR_GRAMMAR)

def eval_with_exception(s):
    with ExpectError():
        return eval(s)
    return False

constrained_expr_grammar.update(
    {
        "<start>": [("<expr>", opts(post=lambda s: eval_with_exception(s) > 10))]
    }
)

assert is_valid_grammar(constrained_expr_grammar)

In [ ]:
def exp_post_expansion_function(expansion):
    """Return the specified post-expansion function, or None if unspecified"""
    return exp_opt(expansion, 'post')

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def eval_function(self, tree, function):
        symbol, children = tree
        # print("Does", all_terminals(tree), "satisfy", repr(function) + "?")

        assert callable(function)

        args = []
        for (symbol, exp) in children:
            if exp != [] and exp is not None:
                symbol_value = all_terminals((symbol, exp))
                args.append(symbol_value)
                
        result = function(*args)
        if self.log:
            print(repr(function) + repr(tuple(args)), "=", repr(result))

        return result

If the `function` returns the Boolean value `False`, the expansion is deemed invalid; and the fuzzer attempts to create another expansion.

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def find_expansion(self, tree):
        symbol, children = tree

        applied_expansion = \
            "".join([child_symbol for child_symbol, _ in children])

        for expansion in self.grammar[symbol]:
            if exp_string(expansion) == applied_expansion:
                return expansion
            
        raise KeyError(symbol + ": did not find expansion" + applied_expansion)
    
    # Return True iff all constraints of grammar are satisfied in TREE
    def run_post_functions(self, tree, depth=float("inf")):
        symbol, children = tree
        expansion = self.find_expansion(tree)

        function = exp_post_expansion_function(expansion)
        if function is None:
            return None

        result = self.eval_function(tree, function)
        if isinstance(result, bool) and not result:
            if self.log:
                print(all_terminals(tree), "did not satisfy", symbol, "constraint")
            return False

        children = self.apply_result(result, children)

        if depth > 0:
            for c in children:
                result = self.run_post_functions(c, depth - 1)
                if isinstance(result, bool) and not result:
                    return False

        return result

The simplest method to check constraints is to retain only those trees that satisfy them.  Works, but can be very slow.

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def fuzz_tree(self):
        while True:
            tree = super().fuzz_tree()
            result = self.run_post_functions(tree)
            if not isinstance(result, bool) or result:
                return tree

In [ ]:
from ExpectError import ExpectError, ExpectTimeout

In [ ]:
constraint_grammar_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
with ExpectTimeout(1):
    expr = constraint_grammar_fuzzer.fuzz()
expr

In [ ]:
with ExpectError():
    eval(expr)

In [ ]:
constrained_expr_grammar.update(
    {
     "<start>": ["<expr>"],
     "<factor>": [
         "+<factor>",
         "-<factor>",
         "(<expr>)",
         ("<integer>.<integer>", opts(post=lambda s1, s2: float(s1 + "." + s2) > 10)),
         ("<integer>", opts(post=lambda n: int(n) > 10))
        ],
    }
)

In [ ]:
from Timer import Timer

In [ ]:
constraint_grammar_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
with Timer() as timer:
    print([constraint_grammar_fuzzer.fuzz() for i in range(10)])

In [ ]:
timer.elapsed_time()

## Checking Elements Sooner

Make things faster: Check as soon as (sub)tree is complete

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def expand_tree_once(self, tree):
        new_tree = super().expand_tree_once(tree)
        
        (symbol, children) = new_tree
        if all([exp_post_expansion_function(expansion) is None for expansion in self.grammar[symbol]]):
            # No constraints for this symbol
            return new_tree
                
        if self.any_possible_expansions(tree):
            # Still expanding
            return new_tree

        result = self.run_post_functions(new_tree, depth=0)
        if not isinstance(result, bool) or result:
            # No constraints, or constraint satisfied
            children = self.apply_result(result, children)
            new_tree = (symbol, children)
            return new_tree

        # Replace tree by unexpanded symbol and try again
        if self.log:
            print(all_terminals(new_tree), "did not satisfy", symbol, "constraint")
            
        if self.replacement_attempts_counter > 0:
            if self.log:
                print("Trying another expansion")
            self.replacement_attempts_counter -= 1
            return (symbol, None)
        
        if self.log:
            print("Starting from scratch")
        raise RestartExpansionException

In [ ]:
class RestartExpansionException(Exception):
    pass

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def __init__(self, grammar, replacement_attempts=10, **kwargs):
        super().__init__(grammar, **kwargs)
        self.replacement_attempts = replacement_attempts

    def fuzz_tree(self):
        while True:
            self.replacement_attempts_counter = self.replacement_attempts
            try:
                tree = super().fuzz_tree()
                return tree
            except RestartExpansionException:
                continue

In [ ]:
constraint_grammar_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
with Timer() as timer:
    print([constraint_grammar_fuzzer.fuzz() for i in range(10)])

In [ ]:
timer.elapsed_time()

## Ordering Expansions

In [ ]:
from IPython.core.display import HTML, display

In [ ]:
from GrammarFuzzer import display_tree

In [ ]:
def exp_order(expansion):
    """Return the specified expansion ordering, or None if unspecified"""
    return exp_opt(expansion, 'order')

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def choose_tree_expansion(self, tree, expandable_children):
        """Return index of subtree in `children` to be selected for expansion.  Defaults to random."""
        (symbol, tree_children) = tree
        if len(expandable_children) == 1:
            # No choice
            return super().choose_tree_expansion(tree, expandable_children)

        expansion = self.find_expansion(tree)
        given_order = exp_order(expansion)
        if given_order is None:
            # No order specified
            return super().choose_tree_expansion(tree, expandable_children)

        nonterminal_children = [c for c in tree_children if c[1] != []]
        assert len(nonterminal_children) == len(given_order), "Order must have one element for each nonterminal"

        # print("Checking ", expandable_children, "against", nonterminal_children, repr(given_order))

        # Find expandable child with lowest ordering
        min_given_order = None
        j = 0
        for k, expandable_child in enumerate(expandable_children):
            while j < len(nonterminal_children) and expandable_child != nonterminal_children[j]:
                j += 1
            assert j < len(nonterminal_children), "Expandable child not found"
            if self.log:
                print("Expandable child #%d %s has order %d" % (k, expandable_child[0], given_order[j]))

            if min_given_order is None or given_order[j] < min_given_order:
                min_given_order = k

        assert min_given_order is not None
        
        if self.log:
            print("Returning expandable child #%d %s" % 
                  (min_given_order, expandable_children[min_given_order][0]))

        return min_given_order

## Usage Examples

### Matching Tags

In [ ]:
from Grammars import crange

In [ ]:
XML_GRAMMAR = {
    "<start>": ["<xml-tree>"],
    "<xml-tree>": ["<<id>><xml-content></<id>>"],
    "<xml-content>": ["Text", "<xml-tree>"],
    "<id>": ["<letter>", "<id><letter>"],
    "<letter>": crange('a', 'z')
}

assert is_valid_grammar(XML_GRAMMAR)

In [ ]:
xml_fuzzer = GrammarFuzzer(XML_GRAMMAR)
xml_fuzzer.fuzz()

In [ ]:
XML_GRAMMAR.update({
    "<xml-tree>": [("<<id>><xml-content></<id>>",
                    opts(post=lambda id1, content, id2: [None, None, id1])
                   )]
})

In [ ]:
xml_fuzzer = ConstraintGrammarFuzzer(XML_GRAMMAR)
xml_fuzzer.fuzz()

### Checksums

In [ ]:
CREDIT_CARD_GRAMMAR = {
    "<start>": ["<credit-card-number>"],
    "<credit-card-number>": [("<digits>", opts(post=fix_luhn_checksum))],
    "<digits>": ["<digit-block><digit-block><digit-block><digit-block>"],
    "<digit-block>": ["<digit><digit><digit><digit>"],
    "<digit>": crange('0', '9')
}

assert is_valid_grammar(CREDIT_CARD_GRAMMAR)

In [ ]:
g = GrammarFuzzer(CREDIT_CARD_GRAMMAR)
cc_number = g.fuzz()
cc_number

In [ ]:
valid_luhn_checksum(cc_number)

In [ ]:
fixed_cc_number = fix_luhn_checksum(cc_number)
fixed_cc_number

In [ ]:
valid_luhn_checksum(fixed_cc_number)

In [ ]:
fixing_fuzzer = ConstraintGrammarFuzzer(CREDIT_CARD_GRAMMAR)
cc_number = fixing_fuzzer.fuzz()
cc_number

In [ ]:
valid_luhn_checksum(cc_number)

### Defining and Using Identifiers

In [ ]:
from Parser import VAR_GRAMMAR

In [ ]:
g = GrammarFuzzer(VAR_GRAMMAR)
for i in range(10):
    print(g.fuzz())

In [ ]:
SYMBOL_TABLE = set()

In [ ]:
def define_id(id):
    SYMBOL_TABLE.add(id)

In [ ]:
def use_id():
    if len(SYMBOL_TABLE) == 0:
        return False

    id = random.choice(list(SYMBOL_TABLE))
    return id

In [ ]:
def clear_symbol_table():
    global SYMBOL_TABLE
    SYMBOL_TABLE = set()

In [ ]:
CONSTRAINED_VAR_GRAMMAR = copy.deepcopy(VAR_GRAMMAR)
CONSTRAINED_VAR_GRAMMAR.update({
    "<start>": [("<statements>", opts(pre=lambda: clear_symbol_table()))],
    "<assignment>": [("<identifier>=<expr>", opts(post=lambda id, expr: define_id(id),
                                                  order=(1, 2)))],
    "<factor>": ['+<factor>', '-<factor>', '(<expr>)',
                 ("<identifier>", opts(post=lambda _: use_id())),
                 '<number>'],
    "<statements>": [("<statement>;<statements>", opts(order=(1, 2))),
                      "<statement>"]
})

assert is_valid_grammar(CONSTRAINED_VAR_GRAMMAR)

In [ ]:
from ExpectError import ExpectTimeout

In [ ]:
g = ConstraintGrammarFuzzer(CONSTRAINED_VAR_GRAMMAR)
for i in range(10):
    print(g.fuzz())

## All Together

In [ ]:
from ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer, ProbabilisticGrammarCoverageFuzzer

In [ ]:
class BigFatGrammarFuzzer(ProbabilisticGrammarFuzzer, ConstraintGrammarFuzzer):
    pass

In [ ]:
class BigFatGrammarCoverageFuzzer(ProbabilisticGrammarCoverageFuzzer, ConstraintGrammarFuzzer):
    pass

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

1. Implement a syntax that allows people to refer to subtrees – say $1.$2 is the second child of the first symbol.


### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

In [ ]:
ATTR_GRAMMAR = {
 "<clause>": ("<xml-open>Text<xml-close>", opts(constraint=lambda x1, x2: x1.name == x2.name)),
 "<xml-open>": ("<langle><tag><rangle>", opts(producer=lambda: (None, opts(name=random_name()), None))),
 "<xml-close>": ("<langle>/<tag><rangle>", opts(producer=lambda: (None, find_name(), None))),
}

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_